# ESP32 AutoGenerate Configurations from CSV

This notebook will focus on the ESP32 configuration generation for us with the ESPHome integration within Home Assistant.

Date of Project: July 24 2022
Software and Devices Used: 
- Home Assistant 2022.7
- Miflora Xiaomi BLE Plant Sensor
- ESP32-Wroom-RevD

Further notebooks will be created to document the other portions of this project which will be available on my Github repositoriy located here.

https://github.com/netmanchris/HassPlants



# Miflora BLE Plant Sensor + ESP32 Config Generation for Home Assistant

Like most things, what was easy at low volume can start to show challenges when reaching certain scales. 

This project is intended to help automate the creation of my home assistant based Plant monitoring setup. This setup uses a combination of the Xiaomi Miflora BLE Plant sensors, ESP32wroom RevD boards, and Home Assistant. 

This has been a work in progress and will likely change as I experiment. 

## WHY

### Why part 1

I killed plants. I'd like my plants to not die.

### Why part 2

Why not? Home Automation is supposed to help us and make our lives easier. This project will allow me to know exactly when any given plant in my home needs water and/or fertilization. I'm not wandering around the house touching plants to see if the it's dry or using a plant monitoring spike from plant to plant to check the soil fertilization. 


When I started this project, it was possible to integrate the Miflora Sensor suing the native BLE Home Assistant Integration, but as of 2022.7 release, the bluethooth libraries that Home Assistant used are no longer usable which will break existing MiFlora integrations using the BLE, and makes BLE no longer an option moving forward. 

![image](./images/20227BLEBroke.png)

So moving forward, the ESPHome based is the easiest of the available options.

In addition to the "It's the easiest way" reason, the other major reason to use the ESPHome based integration is that it allows you to esaily overcome the distance limitations that come along with Bluetooth as a networking technology. 

### Simple Solution

For users wanting to leverage the simple ESPHome based solution, this can be easily done manually and allows you to deploy this solution in 

- Scan the BLE mac-address of the MiFlora sensor using an android-based device (iOS devices don't show the mac-address at the time of this writting)
- Connect the ESP32 to the ESPHome interface in Home Assistant and add the Miflora specific configuration using the mac-address detected in step 1.
- Manually configurat the YAML definition for the Plant integration using the Sensors entities assigned in step 2.
- Add the new plant entities to dashboards

![image](./images/singlesensor.png)

### Scaled Solution

When the same four steps above need to be performed at larger scales, it quickly became obvious to me that I needed to do something to automate these steps. Although this process can be useful to automate the creation of the various configuration files when dealing with just a few sensors, the majority of the work of this project was performed to deal with the a setup with the following attributes

- 1 x home
- 7 x rooms
- 9 x Esp32-Wroom Rev D
- ~80 x Miflora Plant Sensors

![image](./images/multiplesensor.png)






### CSV File Formating and Input Generation

There are currently two files which are required as data inputs for this project

- plants_worksheet.csv is the file where the plants to miflora sensor to esp32 bindings will be performed

Other files in this repository will be used for other portions of this project. Only the plants_worksheet.csv is necessary for the Generation of the ESP32 configurations.

### ESPHome Config Generation

Because of the scale of the amount of sensors I'm using, I needed way to quickly generate the configurations for the ESP32s that could be used in the ESPHome integration to bind a specific Miflora sensor to a the appropriate ESP32.  

The easiest way in my environment to do that was to split the plant sensors in groupings based on rooms ( see the plant CSV file for the entire data structure )

# Plants Worksheet CSV

The Plants worksheet is a CSV which contains all of the information required to generate the ESP32 configurations, plant integration configurations, as well as the dashboards for both desktop/tablet and mobile formats.

The plant worksheet can be located here.

Each row of this worksheet represents a single plant with the following attributes\

- name: Unique Value. One per plant. type: String
- plant_type: One per plant. Must match values in the *plant_attributes.csv* worksheet type: String
- room: Grouping mechanism. Plants with the same value will be automatically grouped together.
- sensor: Unique Value. One per sensor. type: Integer
- sensor_number: Unique Value. One per sensor. This will be appended in the ESP config as the unique portion of the auto-generated sensor entity.  type: String
- sensor_mac: the 12 digit mac-address in the format XX:XX:XX:XX:XX:XX example C4:7C:8D:6D:8D:F6 type: String
- esp_parent: {FUTURE USE} Intention is to use this to create a permanent binding between a specific plant sensor and a specific ESP32 device to prevent naming conflicts when dealing with larger rooms ( > 13 plants per room ) type: String 
- esp_current: Unused. Documentation purposes only. type: String
- room_picture: {FUTURE USE} Intention is to use this to bind specific plants to specific room pictures for building - out dashboards that require multiple camera angles. type: String
- top: Numerical value used a the X coodinate for plant placement for the room picture generation. type: Integer
- left: Numerical value used a the Y coodinate for plant placement for the room picture generation. type: Integer
- flash_light: full name of the script entitiy used to flash the lights where a specific plant is located. type: String






# Templates for ESP32

In this step the goal is to create a jinj2 template file which will generate an ESP32 configuration to bind selected Miflora sensors for use with home assistant. 

Using the ESP_Home component in Home Assistant will automatically create sensors within Home Assistant for each of the sensors exposed by the MiFlora BLE Plant Sensor

Requirements

- load all plants data file
- filter plants for a specific room to a new data frame
- count the number of plants in the data frame
- split the data frame by 13 (see note below)
- create an ESP32 output file 

note ( current observed max number of miflora BLE sensors that can be monitored by an ESP32 wroom-D rev is 13. This number may be different based on newer, more powerfull models of the ESP32. Needs to be revisted with different models ).


**Note**  Might want to allow filtering based on multiple rooms to deal with situations where plants from more than one room are bound to a single ESP32.  ( Ie. My Kitchen and Dining Room have less than 13 plants total, so I'm able to consolidate those two rooms on to a single ESP32 wroom-revD )

## ESP32 Jinja2 Template

Thing to note in this template

A max of 13 sensors will be bound to a single ESP32 Wroom Rev d.  In testing this is the maximum amount of sensors configuration before things go weird. 14 may allow the ESP to run, but it seems to crash after some period of time. 15 sensors will not allow the ESP to boot up.

For the naming convention

I've used the name. "plant_{{plant_sensor}}..." for my naming convention. This will give us a predictable, human readable value for the entity ids that will show up in home assistant.

for instance, a sensor called  "s12" will have the following home assistant entities automatically created which can be used later in the plant integration definition.

```
sensor.plant_s12_temperature
sensor.plant_s12_moisture
sensor.plant_s12_illuminance
sensor.plant_s12_soil_conductivity
sensor.plant_s12_battery_level
```

Note: The battery level is no longer reported through passive bluetooth by the Miflora sensors, I've left it in here as it doesn't break anything with no devices and may still be useful for sensors with older firmware on them. 



```
{% for row in plants|batch(13) %}    #ESP32-Wroom-RevD can have a max of 13 sensors defined before things go weird.

esp32_ble_tracker:

sensor:
{% for plant in row %}
  - platform: xiaomi_hhccjcy01
    mac_address: "{{plant.sensor_mac}}"     #Pulled directly from the plants_worksheet.csv file
    temperature:
      name: "plant_{{plant.sensor}}_temperature"    #pulled directly from the plants_worksheet.csv file
    moisture:
      name: "plant_{{plant.sensor}}_moisture"
    illuminance:
      name: "plant_{{plant.sensor}}_illuminance"
    conductivity:
      name: "plant_{{plant.sensor}}_soil_conductivity"
    battery_level:
      name: "plant_{{plant.sensor}}_battery_level"

{% endfor %}{% endfor %}

```

# Generating the Configuration Files

In this section, we'll go through the python code used to generate the actual configurations


In [12]:
import pandas as pd
from jinja2 import Template
import os

# Load Plants CSV

In this section, we will load the plants CSV file which will be used to create the ESPHome configurations to bind the Miflora BLE Plant sensors to a specific ESP32. 

There is one file needed here.

1. plants_worksheet.csv which includes the various plants, sensors, room locations, corresponding script to flash lights.






In [13]:
#plants_df = pd.read_csv('data/plants_worksheet.csv', index_col=0)
plants_df = pd.read_csv('data/, index_col=0)



FileNotFoundError: [Errno 2] No such file or directory: 'data/plants_worksheet_sample.csv'

## Create Rooms List

In this step, we will create a list of the room names defined in the CSV file. Defining the room names logically is important in that ESP32 configurations, Plant entity definitions, and dashboards for each individual room will be automatically generated through this code. Spending some time thinking about how you want to organize your plants dashboards is a very, very good idea.

Note: I've got logic in the jinja2 files that will check to see whether the element in the room list is of type "string" which will allow sensors unassigned to rooms to be ignored. 

TODO - CHECK TO MAKE SURE THIS IS TRUE FOR ALL TEMPLATES

In [4]:
rooms_list = list(plants_df.room.unique())
rooms_list

['boys_bedroom',
 'dining_room',
 'downstairs_bathroom',
 'games_room',
 'home_office',
 'kitchen',
 'living_room',
 'master_bedroom',
 nan]

## Loading the ESP32 Jinja2 Templates

In this step, we load the ESP specific templates from the templates folder and load it as a jinja2 template.

- ESP32 configurations to be cut and paste into a specific ESP32 device.  * Note - It is recommended that you use at least an ESP32wroom revD. In my testing, earlier version of the ESP32 weren't able to detect the BLE beacons from the Miflora.

Note: Currently the ESP template will generate one file per room. This file will be split into sections which create a configuration for up to 13 Miflora sensors per section.



In [10]:
with open('templates/esp.j2') as f:
    esp_template = f.read()
j2_esp_template = Template(esp_template)

## Rendering ESP Configurations

We will now render the ESP32 Configurations.

Note: As a prerequisite, the directory structure has to preexist to be able to properly save the file configurations to disk. 




In [11]:
for room in rooms_list:
    if type(room) is float:
        pass
    room = [room]
    room_df = plants_df[plants_df.room.isin(room)]
    #print (len(room_df))
    if len(room_df) > 1:
        #print (room)
        esp_df = room_df[room_df.sensor_model.eq('Miflora')]
        room_dict = esp_df.to_dict('records')
        #print (len(room_dict))
        
        # OUTPUT THE ESP32 ESPHome Configuration
        with open('output/esp/'+room_dict[0]['room']+'.txt', 'w') as f:
            f.write(j2_esp_template.render(plants=room_dict))

# Sample Output

You can check the ./output/esp/ folder for sample output where you should have one file for each room with the configuration as follows

```

esp32_ble_tracker:

sensor:

  - platform: xiaomi_hhccjcy01
    mac_address: "C4:7C:8D:6D:8D:F6"
    temperature:
      name: "plant_s7_temperature"
    moisture:
      name: "plant_s7_moisture"
    illuminance:
      name: "plant_s7_illuminance"
    conductivity:
      name: "plant_s7_soil_conductivity"
    battery_level:
      name: "plant_s7_battery_level"
      
```

This configuration can be cut and paste into the ESPHome configuration for a specific ESP32 which you've attached to your home assistant instance as shown below


![ESPHomeConfig.png](./images/ESPHomeConfig.png)


Once you have accepted the ESP32 into your Home Assistant Integrations You can verify that the sensor has been created with the correct entity names.

![ESP32_dr_sample.png](./images/esp32_dr_sample.png)

You can also verify the specific entities if you'd like

![sensor_s7_entities.png](./images/sensor_s7_entities.png)




# Things to Note

You'll want to be careful when re running this multiple times as, depending on the order of the plant sensors in the CSV file, the specific 13 sensors that will be allocated to a specific ESP32 may not be the same each time you run this.

If you have a single ESP32 per room, the order shouldn't matter, but if you have multiple ESP32's dedicated to a single room because of the amount of plants in that room, you may accidently move the sensors from one ESP32 to a second.

This will result in Home Assistant automatically appending a "_1" or "_2" to the end of the entity name to ensure a unique entity ID in the home assistant database.  

This will break the rest of the templates.  

The easiest way to deal with this would be to completely remove the ESP32 (delete the integration for that specific ESP32) and then perform a full reboot of Home Assistant to clear any remaining cache. 

Although it's possible to manually remove the sensors, it can be a slightly more painful procedure. Because the configurations are all automatically generated, I've found it easier to simply remove and re-add the ESP32s for a specific room.

